In [18]:
import pandas as pd
import re

In [19]:
df=pd.read_csv('llm.csv')

In [20]:
print(df.head())

   index                                         image_link  group_id  \
0      0  https://m.media-amazon.com/images/I/110EibNycl...    156839   
1      1  https://m.media-amazon.com/images/I/11TU2clswz...    792578   
2      2  https://m.media-amazon.com/images/I/11TU2clswz...    792578   
3      3  https://m.media-amazon.com/images/I/11TU2clswz...    792578   
4      4  https://m.media-amazon.com/images/I/11gHj8dhhr...    792578   

  entity_name                                         prediction  
0      height                                           65-190cm  
1       width                                                 13  
2      height  Sorry, as a base VLM I am not trained to answe...  
3       depth                                                 13  
4       depth                                                110  


In [21]:
pred=df['prediction']

In [22]:
UNIT_CORRECTIONS = {
    'g': 'gram',
    'gr': 'gram',
    'gm': 'gram',
    'gms': 'gram',
    'kg': 'kilogram',
    'kgs': 'kilogram',
    'mg': 'milligram',
    'mgs': 'milligram',
    'μg': 'microgram',
    'mcg': 'microgram',
    'oz': 'ounce',
    'lb': 'pound',
    'lbs': 'pound',
    'ton': 'ton',
    'ml': 'millilitre',
    'mL': 'millilitre',
    'microl': 'microlitre',
    'litre': 'litre',
    'liter': 'litre',
    'litres': 'litre',
    'l': 'litre',
    'ltr': 'litre',
    'cl': 'centilitre',
    'dl': 'decilitre',
    'floz': 'fluid ounce',
    'fluidounce': 'fluid ounce',
    'fluidoz': 'fluid ounce',
    'gal': 'gallon',
    'imperialgallon': 'imperial gallon',
    'cup': 'cup',
    'pint': 'pint',
    'qt': 'quart',
    'quart': 'quart',
    'cubicfoot': 'cubic foot',
    'cubicinch': 'cubic inch',
    '"': 'inch',
    'inches': 'inch',
    'inch': 'inch',
    'in': 'inch',
    'cm': 'centimetre',
    'CM': 'centimetre',
    'centimetre': 'centimetre',
    'centimeter': 'centimetre',
    'mm': 'millimetre',
    'millimetre': 'millimetre',
    'millimeter': 'millimetre',
    'm': 'metre',
    'meter': 'metre',
    'metre': 'metre',
    'ft': 'foot',
    'feet': 'foot',
    'yd': 'yard',
    'yards': 'yard',
    'v': 'volt',
    'V': 'volt',
    'vv': 'volt',
    'volt': 'volt',
    'kv': 'kilovolt',
    'mv': 'millivolt',
    'w': 'watt',
    'W': 'watt',
    'watt': 'watt',
    'kw': 'kilowatt',
    'sqft': 'cubic foot',
    'sqin': 'cubic inch',
    'pounds': 'pound',
    'incm': 'inch',
    'gallon': 'gallon',
    'grams': 'gram',
    'ounces': 'ounce',
    'pound': 'pound',
    'gsm': 'gram',
    'tons': 'ton',
    'centimetre': 'centimetre',
    'foot': 'foot',
    'inch': 'inch',
    'metre': 'metre',
    'millimetre': 'millimetre',
    'yard': 'yard',
    'gram': 'gram',
    'kilogram': 'kilogram',
    'microgram': 'microgram',
    'milligram': 'milligram',
    'ounce': 'ounce',
    'pound': 'pound',
    'ton': 'ton',
    'kilovolt': 'kilovolt',
    'millivolt': 'millivolt',
    'volt': 'volt',
    'kilowatt': 'kilowatt',
    'watt': 'watt',
    'centilitre': 'centilitre',
    'cubic foot': 'cubic foot',
    'cubic inch': 'cubic inch',
    'cup': 'cup',
    'decilitre': 'decilitre',
    'fluid ounce': 'fluid ounce',
    'gallon': 'gallon',
    'imperial gallon': 'imperial gallon',
    'litre': 'litre',
    'microlitre': 'microlitre',
    'millilitre': 'millilitre',
    'pint': 'pint',
    'quart': 'quart',
}



def extract_numeric_and_suffix(text):
    match = re.search(r'\d.*', text)
    return match.group(0) if match else ''

#AC100-277V
def value(text):
    text=str(text)
    text=extract_numeric_and_suffix(text)
    result=''
    for i in str(text):
        if i.isnumeric() or i=='.' or i==',' or i=='/':
            result+=i
        else:
            break
    

    match = re.search(r'(\d+)\s*(\d+)/(\d+)', text)
    if match:
        whole_number = int(match.group(1))
        numerator = int(match.group(2))
        denominator = int(match.group(3))
        decimal_fraction = numerator / denominator
        result = whole_number + decimal_fraction
        result=str(result)
        
    if ',' in result:
        if result.count(',') == 1 and result.count('.') == 0:
            # Check if the comma is in the last 3 digits (likely a thousands separator)
            if len(result.split(',')[-1]) <= 2:  
                result = result.replace(',', '.')  
            else:
                result = result.replace(',', '')   
        else:
            result = result.replace(',', '')  

    return result

def unit(text):
    text=str(text)
    count=0
    text=extract_numeric_and_suffix(text)
    unit=''
    for i in str(text):
        if i.isnumeric() or i=='.' or i==' ' or i==',' or i=='-' or i=='x' or i=='*':
            pass
        else:
            if i=='(':
                break
            if i=='/' and count>=1:
                break
            elif i=='/' and count==0:
                continue
            unit+=i
            count+=1
    #make unit is small and then check if it is in the allowed units       
    unit = unit.lower()
    if unit in UNIT_CORRECTIONS:
        unit=UNIT_CORRECTIONS[unit]
        return unit
    return ''

In [23]:
#create a new column 'value' and apply the function 'unit' to the column 'prediction'
df['value'] = pred.apply(value)
df['unit'] = pred.apply(unit)

#lets say value exists but unit does not exist then, we shall put unit as the one that shares similiar group_id
#this will get applied to only empty string units with non-empty value
#create a new column called prediction
#only merge the columns if both each row of the 'value' and 'unit' are not empty
df['prediction'] = df.apply(lambda row: f"{row['value']} {row['unit']}" if row['value'] != '' and row['unit'] != '' else '', axis=1)



In [24]:
#export csv file
df.to_csv('ultimate.csv', index=False)

In [17]:
#print(value('5.5cm (2.2inch)'))
#print(unit('5.5cm (2.2inch)'))

In [25]:
# Load the CSV file
file_path = 'ultimate.csv'  # Replace with your file path
df = pd.read_csv(file_path)

# Define the entity_unit_map
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon',
                    'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Function to validate if the unit in the 'Answer' matches the entity_name
def is_valid_unit(row):
    entity = row['entity_name']
    answer = str(row['prediction']).lower()  # Convert to string and normalize to lowercase
    if entity in entity_unit_map:
        # Check if any unit in the map for this entity appears in the answer
        for unit in entity_unit_map[entity]:
            if unit in answer:
                return True
    return False

# Replace invalid answers with an empty string
df['prediction'] = df.apply(lambda row: row['prediction'] if is_valid_unit(row) else '', axis=1)

# Save the updated DataFrame
output_path = 'ultimate.csv'  # Replace with your desired output path
df.to_csv(output_path, index=False)

print(f"Updated data saved to {output_path}")

C:\Users\dell\AppData\Local\Temp\ipykernel_27736\931399896.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Updated data saved to ultimate.csv
